In [1]:
import re
import numpy as np
import pandas as pd
import pickle

In [2]:
pd.options.display.max_columns=None
pd.options.display.max_rows=100
pd.set_option('display.max_colwidth', None)

In [3]:
df = pd.read_csv('../datos/datos_sin_unos.csv', index_col = 0)
df.head(2)

,Unnamed: 0,age,gender,country,continent,job_title,work_activities,size_DA_dept,years_programming,first_language_rec,dev_language,primary_data_tool,IDE,big_data,BI_tools,visualisation,ML,notebooks_KaggleNotebooks,notebooks_ColabNotebooks,notebooks_AzureNotebooks,notebooks_Paperspace/Gradient,notebooks_Binder/JupyterHub,notebooks_CodeOcean,notebooks_IBMWatsonStudio,notebooks_AmazonSagemakerStudioNotebooks,notebooks_AmazonEMRNotebooks,notebooks_GoogleCloudNotebooks(AIPlatform/VertexAI),notebooks_GoogleCloudDatalab,notebooks_DatabricksCollaborativeNotebooks,notebooks_Zeppelin/ZeplNotebooks,notebooks_DeepnoteNotebooks,notebooks_ObservableNotebooks,notebooks_None,notebooks_Other,sharing_PlotlyDash,sharing_Streamlit,sharing_NBViewer,sharing_GitHub,sharing_Personalblog,sharing_Kaggle,sharing_Colab,sharing_Shiny,sharing_does_not_share,sharing_Other,dev_language_Javascript,dev_language_None,dev_language_R,dev_language_Bash,dev_language_Python,dev_language_Other,dev_language_C,dev_language_MATLAB,dev_language_Julia,dev_language_Java,dev_language_C++,dev_language_Swift,dev_language_SQL,IDE_None,IDE_Sublime Text,IDE_Vim / Emacs,IDE_RStudio,IDE_Jupyter (JupyterLab; Jupyter Notebooks; etc),IDE_Other,IDE_Visual Studio,IDE_Notepad++,IDE_Visual Studio Code (VSCode),IDE_Spyder,IDE_PyCharm,IDE_MATLAB,IDE_Jupyter Notebook,visualisation_None,visualisation_Matplotlib,visualisation_Other,visualisation_Seaborn,visualisation_Altair,visualisation_Shiny,visualisation_D3 js,visualisation_Bokeh,visualisation_Leaflet / Folium,visualisation_Plotly / Plotly Express,visualisation_Geoplotlib,visualisation_Ggplot / ggplot2,activities_ML_prototypes,activities_run_ML,activities_ML_research,activities_Other,activities_improve_ML,activities_None,activities_analyze_data,activities_data_infrastructure,big_data_Google Cloud Firestore,big_data_Google Cloud BigTable,big_data_Google Cloud Spanner,big_data_PostgreSQL,big_data_Microsoft Azure Cosmos DB,big_data_SQLite,big_data_Amazon DynamoDB,big_data_None,big_data_Microsoft Azure SQL Database,big_data_Amazon RDS,big_data_Other,big_data_Snowflake,big_data_Amazon Aurora,big_data_Google Cloud SQL,big_data_Microsoft SQL Server,big_data_Oracle Database,big_data_Google Cloud BigQuery,big_data_Amazon Redshift,big_data_IBM Db2,big_data_MongoDB,big_data_MySQL,BI_tools_None,BI_tools_Tableau,BI_tools_Looker,BI_tools_Google Data Studio,BI_tools_TIBCO Spotfire,BI_tools_Other,BI_tools_Qlik,BI_tools_Tableau CRM,BI_tools_SAP Analytics Cloud,BI_tools_Sisense,BI_tools_Microsoft Power BI,BI_tools_Amazon QuickSight,BI_tools_Salesforce,BI_tools_Microsoft Azure Synapse,BI_tools_Domo,BI_tools_Thoughtspot,BI_tools_Alteryx
0,0,50-54,Man,India,Asia,Other,None of these activities are an important part of my role at work,3-4,5-10 years,Python,"Python, R","Local development environments (RStudio, JupyterLab, etc.)",Vim / Emacs,PostgreSQL,None,"Matplotlib ,Seaborn ,Ggplot / ggplot2 ,Shiny ,Leaflet / Folium",5-10 years,NaN,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GitHub,NaN,Kaggle,NaN,NaN,NaN,NaN,NaN,NaN,R,NaN,Python,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vim / Emacs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matplotlib,NaN,Seaborn,NaN,Shiny,NaN,NaN,Leaflet / Folium,NaN,NaN,Ggplot / ggplot2,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,PostgreSQL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,50-54,Man,Indonesia,Asia,Program/Project Manager,Build and/or run the data infrastructure that my business uses for storing; analyzing; and operationalizing data,1-2,20+ years,Python,"SQL, C, C++, Java","Advanced statistical software (SPSS, SAS, etc.)","Notepad++,Jupyter Notebook",NaN,NaN,Matplotlib,< 1 year,Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,NaN,NaN,Java,C++,NaN,SQL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Notepad+

In [4]:
df_da = df[(df["job_title"] == "Business Analyst") | (df["job_title"] == "Data Analyst")]

In [5]:
with open('../datos/diccionario_respuestas.pkl', 'rb') as f:
    diccionario_respuestas = pickle.load(f)

In [6]:
lista_columnas_combinar = ['dev_language_', 'IDE_', 'visualisation_', 'activities_', 'big_data_', 'BI_tools_', 'sharing_', 'notebooks_']

In [7]:
df_da.replace(np.nan, "", inplace=True)

/tmp/ipykernel_760/3257084013.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_da.replace(np.nan, "", inplace=True)


In [8]:
def replace_punto_coma(col):
    return re.sub(r";+", ";", col)

In [9]:
def replace_none(col):
    return re.sub(r"None;", ";", col)

In [10]:
def columna_combo(lista_columnas, nombre):

    df_da[nombre] = ""

    for col in df_da.columns:
        for i in lista_columnas:
            if i in col:
                df_da[nombre] += df_da[col] + ";"

    df_da[nombre] = df_da[nombre].apply(replace_punto_coma)
    
    df_da[nombre] = df_da[nombre].apply(replace_none)

    df_da[nombre] = df_da[nombre].apply(lambda x: x.strip(";"))
    
    df_da[nombre] = df_da[nombre].apply(replace_punto_coma)

    df_combo_counts = df_da[nombre].value_counts().reset_index()

    df_combo_counts.columns = ['tools', 'count']
    
    display(df_combo_counts.head(20))
    
    return df_combo_counts

In [11]:
df_combo_all = columna_combo(lista_columnas_combinar, 'combo_all')

/tmp/ipykernel_760/3071236157.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_da[nombre] = ""
/tmp/ipykernel_760/3071236157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_da[nombre] = ""
/tmp/ipykernel_760/3071236157.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

,tools,count
0,analyze_data,136
1,,37
2,Other,21
3,analyze_data;data_infrastructure,16
4,data_infrastructure,11
5,ML_prototypes;run_ML;ML_research;improve_ML;analyze_data;data_infrastructure,8
6,ML_research;analyze_data,7
7,run_ML,5
8,Kaggle Notebooks;Python;Jupyter Notebook;Matplotlib;Seaborn,4
9,run_ML;analyze_data,4


In [12]:
df_dev_language_vis = columna_combo(['dev_language_', 'visualisation_'], 'combo_dev_lang_vis')

/tmp/ipykernel_760/3071236157.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_da[nombre] = ""
/tmp/ipykernel_760/3071236157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_da[nombre] = ""
/tmp/ipykernel_760/3071236157.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

,tools,count
0,,314
1,Python;SQL;Matplotlib;Seaborn,223
2,Python;Matplotlib;Seaborn,154
3,Python;SQL;Matplotlib;Seaborn;Plotly / Plotly Express,94
4,Python,72
5,Python;SQL;Matplotlib,72
6,Python;Matplotlib,72
7,SQL,69
8,Python;SQL,54
9,Python;Matplotlib;Seaborn;Plotly / Plotly Express,52


In [13]:
#['dev_language_', 'IDE_', 'visualisation_', 'activities_', 'big_data_', 'BI_tools_', 'sharing_', 'notebooks_']

In [14]:
df_dev_language_BI = columna_combo(['dev_language_', 'BI_tools_'], 'combo_dev_lang_BI')

/tmp/ipykernel_760/3071236157.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_da[nombre] = ""
/tmp/ipykernel_760/3071236157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_da[nombre] = ""
/tmp/ipykernel_760/3071236157.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

,tools,count
0,Python,326
1,Python;SQL,314
2,,311
3,R;Python;SQL,152
4,R;Python,86
5,Python;SQL;Tableau,62
6,Python;SQL;Microsoft Power BI,59
7,SQL,55
8,Python;SQL;Tableau;Microsoft Power BI,45
9,Python;Tableau,40


In [15]:
df_dev_language_big_data = columna_combo(['dev_language_', 'big_data_'], 'combo_dev_lang_big_data')

/tmp/ipykernel_760/3071236157.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_da[nombre] = ""
/tmp/ipykernel_760/3071236157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_da[nombre] = ""
/tmp/ipykernel_760/3071236157.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

,tools,count
0,Python,328
1,,321
2,Python;SQL,293
3,R;Python;SQL,144
4,R;Python,94
5,SQL,54
6,R,39
7,Python;SQL;MySQL,37
8,Python;MySQL,33
9,Javascript;Python;Java;SQL,31


In [16]:
df_dev_language_notebooks = columna_combo(['dev_language_', 'notebooks_'], 'combo_dev_lang_notebooks_')

/tmp/ipykernel_760/3071236157.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_da[nombre] = ""
/tmp/ipykernel_760/3071236157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_da[nombre] = ""
/tmp/ipykernel_760/3071236157.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

,tools,count
0,,318
1,Python;SQL;Python;SQL,179
2,Python;Python,132
3,Kaggle Notebooks;Python;SQL; Kaggle Notebooks;Python;SQL,94
4,Colab Notebooks;Python;SQL;Colab Notebooks;Python;SQL,84
5,Kaggle Notebooks;Python; Kaggle Notebooks;Python,78
6,R;Python;SQL;R;Python;SQL,74
7,Kaggle Notebooks;Colab Notebooks;Python;SQL; Kaggle Notebooks;Colab Notebooks;Python;SQL,73
8,Colab Notebooks;Python;Colab Notebooks;Python,72
9,SQL;SQL,66


In [17]:
df_combo_all_tools = columna_combo(['dev_language_', 'IDE_', 'visualisation_', 'big_data_', 'BI_tools_', 'notebooks_'], 'combo_all_tools')

/tmp/ipykernel_760/3071236157.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_da[nombre] = ""
/tmp/ipykernel_760/3071236157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_da[nombre] = ""
/tmp/ipykernel_760/3071236157.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

,tools,count
0,,288
1,R;RStudio;Ggplot / ggplot2;R;R,9
2,Python;Jupyter Notebook;Matplotlib;Seaborn;Python;Python,9
3,Python;SQL;Jupyter Notebook;Matplotlib;Seaborn;Python;SQL;Python;SQL,8
4,Kaggle Notebooks;Colab Notebooks;Python;Jupyter Notebook;Matplotlib;Seaborn; Kaggle Notebooks;Colab Notebooks;Python; Kaggle Notebooks;Colab Notebooks;Python,7
5,Python;PyCharm;Python;Python,7
6,Kaggle Notebooks;Python;Jupyter Notebook;Matplotlib;Seaborn; Kaggle Notebooks;Python; Kaggle Notebooks;Python,6
7,SQL;SQL;SQL,6
8,Kaggle Notebooks;Python;SQL;Jupyter Notebook;Matplotlib;Seaborn; Kaggle Notebooks;Python;SQL; Kaggle Notebooks;Python;SQL,6
9,Other;Other;Other,6


In [18]:
diccionario_respuestas

{'Unnamed: 0': [],
 'age': ['50-54',
  '22-24',
  '45-49',
  '25-29',
  '18-21',
  '30-34',
  '40-44',
  '35-39',
  '70+',
  '55-59',
  '60-69'],
 'gender': ['Man',
  'Woman',
  'Nonbinary',
  'Prefer not to say',
  'Prefer to self-describe'],
 'country': ['Kenya',
  'Algeria',
  'Iraq',
  'Egypt',
  'South Africa',
  'Hong Kong (S.A.R.)',
  'Brazil',
  'Ireland',
  'Romania',
  'Colombia',
  'Ukraine',
  'Portugal',
  'Ecuador',
  'Singapore',
  'Uganda',
  'Australia',
  'Taiwan',
  'Iran',
  'Russia',
  'Tunisia',
  'United Arab Emirates',
  'France',
  'Japan',
  'Pakistan',
  'Switzerland',
  'Philippines',
  'Israel',
  'Thailand',
  'Nepal',
  'Norway',
  'Malaysia',
  'Italy',
  'Kazakhstan',
  'Morocco',
  'Saudi Arabia',
  'Greece',
  'Nigeria',
  'Germany',
  'I do not wish to disclose my location',
  'Belarus',
  'Spain',
  'South Korea',
  'Canada',
  'Other',
  'Islamic Republic of...',
  'Denmark',
  'Czech Republic',
  'Peru',
  'Mexico',
  'Belgium',
  'Ghana',
  'Ethi